In [1]:
from ZV_functions import *
import tensorflow as tf
from matplotlib import pyplot as plt
import matplotlib
import pandas as pd
from keras.layers import Dense, Dropout, LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
from keras.constraints import maxnorm
import keras.backend as k
from tensorflow.keras import regularizers
matplotlib.rcParams.update({'font.size': 16})

Using TensorFlow backend.


In [10]:
#base_dir='/eos/home-a/ahakimi/www/ZV_analysis'
base_dir='D:\\Travail\\ZV_analysis\\'
#year='2018_SR'
years=['2018_SR_var', '2017_SR_var', '2016_SR_var']
name='All_years_SR_allfeats'
cut='Resolved_SR'
vers=''

if cut == 'Boosted_SR':
    features =['Lep_pt1', 'Lep_pt2', 'Lep_eta1',
           'Lep_eta2', 'mll',
           'FatJet_pt', 'FatJeteta',
           'Zlep_1', 'Zlep_2', 
           'vbs_jet_pt1', 'vbs_jet_pt2',
           'vbs_jet_eta1', 'vbs_jet_eta2',
           'mjj_max', 'detajj_mjjmax', 'dphijj_mjjmax', 
            'njet', 'Zvjet', 'vbs_jet_qgl1', 'vbs_jet_qgl2', 'nbtag'
              ]
elif cut == 'Resolved_SR':
    features=['Lep_pt1', 'Lep_pt2', 'Lep_eta1',
           'Lep_eta2', 'mll',
           'Zlep_1', 'Zlep_2', 
           'vbs_jet_pt1', 'vbs_jet_pt2',
           'vbs_jet_eta1', 'vbs_jet_eta2',
           'V_jet_pt1', 'V_jet_pt2',
           'V_jet_eta1', 'V_jet_eta2',
           'mjj_max', 'detajj_mjjmax', 'dphijj_mjjmax',
           'V_jet_mass', 'njet', 'vbs_jet_qgl1', 'vbs_jet_qgl2',
             'V_jet_qgl1', 'V_jet_qgl2', 'nbtag']
"""


if cut == 'Boosted_SR':
    features =['pt1', 'pt2', 'eta1',
           'eta2', 'mll',
           'FatJet_pt', 'FatJeteta',
           'Zlep_1', 'Zlep_2', 
           'vbs_jet_pt1', 'vbs_jet_pt2',
           'vbs_jet_eta1', 'vbs_jet_eta2',
           'mjj_max', 'detajj_mjjmax']
elif cut == 'Resolved_SR':
    features=['pt1', 'pt2', 'eta1',
           'eta2', 'mll',
           'Zlep_1', 'Zlep_2', 
           'vbs_jet_pt1', 'vbs_jet_pt2',
           'vbs_jet_eta1', 'vbs_jet_eta2',
           'V_jet_pt1', 'V_jet_pt2',
           'V_jet_eta1', 'V_jet_eta2',
           'mjj_max', 'detajj_mjjmax',
           'V_jet_mass']
"""
    

"\n\n\nif cut == 'Boosted_SR':\n    features =['pt1', 'pt2', 'eta1',\n           'eta2', 'mll',\n           'FatJet_pt', 'FatJeteta',\n           'Zlep_1', 'Zlep_2', \n           'vbs_jet_pt1', 'vbs_jet_pt2',\n           'vbs_jet_eta1', 'vbs_jet_eta2',\n           'mjj_max', 'detajj_mjjmax']\nelif cut == 'Resolved_SR':\n    features=['pt1', 'pt2', 'eta1',\n           'eta2', 'mll',\n           'Zlep_1', 'Zlep_2', \n           'vbs_jet_pt1', 'vbs_jet_pt2',\n           'vbs_jet_eta1', 'vbs_jet_eta2',\n           'V_jet_pt1', 'V_jet_pt2',\n           'V_jet_eta1', 'V_jet_eta2',\n           'mjj_max', 'detajj_mjjmax',\n           'V_jet_mass']\n"

In [11]:
X,y, variables_list=prep_data_multi(years,name, cut, vers, base_dir) #

xlims=[(0,200),(0,150),(-3,3),(-3,3),(0,150),(0,5),(0,10),(0,900),(
    -5,5),(0,200),(0,1),(-1.5,1.5),(-1.5,1.5),(-1,1),
       (0,200),(0,200),(-5,5),(-5,5),
           (0,200),(0,100),(-5,5),(-5,5),(0,3000),(0,10),(0,4),(0,150)]

#plot_distrib(variables_list, xlims,base_dir, year, cut)
print('NUmber of events: {}, signal : {}, bkg: {}'.format(len(y),len(y[y['signal']==1]),len(y[y['signal']==0])))
print('SOW: {}, signal : {}, bkg: {}'.format(sum(y['weight_']),sum(y['weight_'][y['signal']==1]),sum(y['weight_'][y['signal']==0])))

2018_SR_var
2017_SR_var
2016_SR_var
Resolved_SR_Lep_pt1 114.85145 83.673805
Resolved_SR_Lep_pt2 50.564224 33.075985
Resolved_SR_Lep_eta1 -0.003113448 1.2257481
Resolved_SR_Lep_eta2 -0.005415344 1.2243143
Resolved_SR_mll 90.82381 4.084963
Resolved_SR_Zepp_ll 0.66047484 0.52156
Resolved_SR_nFatJet 1.0360354288402969 1.0576828217491285
Resolved_SR_FatJet_pt -999.0 0.0
Resolved_SR_FatJeteta -999.0 0.0
Resolved_SR_FatJet_softdropmass 0.0 0.0
Resolved_SR_FatJet_tau21 -999.0 0.0
Resolved_SR_Zlep_1 -0.00096163986 0.45851913
Resolved_SR_Zlep_2 -0.0015202983 0.45787966
Resolved_SR_category 1.0 0.0
Resolved_SR_vbs_jet_pt1 173.92517 127.62349
Resolved_SR_vbs_jet_pt2 77.58773 64.77587
Resolved_SR_vbs_jet_eta1 -0.0008007663 1.9228814
Resolved_SR_vbs_jet_eta2 0.0027725587 2.5644786
Resolved_SR_V_jet_pt1 81.38931 53.416916
Resolved_SR_V_jet_pt2 42.688206 13.778909
Resolved_SR_V_jet_eta1 -8.7408036e-05 1.343125
Resolved_SR_V_jet_eta2 0.00103994 1.4282874
Resolved_SR_mjj_max 789.76245 564.7707
Resolved_

D:\Travail\ZV_analysis\ZV_functions.py:242: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['signal']=y['signal'].astype('int32')
D:\Travail\ZV_analysis\ZV_functions.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['w']=0
D:\Travail\ZV_analysis\ZV_functions.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[

saving files
Data prepared, X and y saved to D:\Travail\ZV_analysis\All_years_SR_allfeats\Resolved_SR\data.
NUmber of events: 786365, signal : 73325, bkg: 713040
SOW: 54206.094451577774, signal : 238.05649238883498, bkg: 53968.03795918898


Index(['Resolved_SR_Lep_pt1', 'Resolved_SR_Lep_pt2', 'Resolved_SR_Lep_eta1',
       'Resolved_SR_Lep_eta2', 'Resolved_SR_mll', 'Resolved_SR_Zepp_ll',
       'Resolved_SR_nFatJet', 'Resolved_SR_FatJet_pt', 'Resolved_SR_FatJeteta',
       'Resolved_SR_FatJet_softdropmass', 'Resolved_SR_FatJet_tau21',
       'Resolved_SR_Zlep_1', 'Resolved_SR_Zlep_2', 'Resolved_SR_category',
       'Resolved_SR_vbs_jet_pt1', 'Resolved_SR_vbs_jet_pt2',
       'Resolved_SR_vbs_jet_eta1', 'Resolved_SR_vbs_jet_eta2',
       'Resolved_SR_V_jet_pt1', 'Resolved_SR_V_jet_pt2',
       'Resolved_SR_V_jet_eta1', 'Resolved_SR_V_jet_eta2',
       'Resolved_SR_mjj_max', 'Resolved_SR_detajj_mjjmax',
       'Resolved_SR_dphijj_mjjmax', 'Resolved_SR_V_jet_mass',
       'Resolved_SR_njet', 'Resolved_SR_Zvjet', 'Resolved_SR_vbs_jet_qgl1',
       'Resolved_SR_vbs_jet_qgl2', 'Resolved_SR_V_jet_qgl1',
       'Resolved_SR_V_jet_qgl2', 'Resolved_SR_FatJet_qgl', 'Resolved_SR_nbtag',
       'Resolved_SR_lepton_id'],
      dtype='o

In [5]:

data_dir=base_dir+name+'\\'+cut+'\\data'+vers


input_list= features
inputs= [cut+'_'+i for i in input_list]



X= pd.DataFrame(np.load(data_dir+'\\X_{}_{}{}.npy'.format(name,cut,vers), allow_pickle=True), columns=variables_list)
X=X[inputs]
y= pd.DataFrame(np.load(data_dir+'\\y_{}_{}{}.npy'.format(name,cut,vers), allow_pickle=True), columns= ['year', 'signal', 'sample','group','weight_', 'w'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2, random_state=42)


session_conf = tf.ConfigProto(intra_op_parallelism_threads=12, inter_op_parallelism_threads=12)
tf.set_random_seed(1)
session_conf.gpu_options.allow_growth = True  # dynamically grow the memory 
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
k.tensorflow_backend.set_session(sess)

early_stop=5  #number of epochs without gain before stoping
predictions_NN={}
target={}


In [6]:
def build_dnn(base_dir, lr, l1, l2, dropout, sl1, sl2, sl3, sl4, sl5):
    from keras.layers.normalization import BatchNormalization

    SoB_thr=0.8
   
    
    NN_name='DNN_{}_{}_{}_{}_{}_lr{}_dp{}'.format(sl1,sl2,sl3,sl4,sl5,lr,dropout)
    reg=regularizers.L1L2(l1,l2)
    opt=Adam(lr=lr)
    #opt=SGD(lr=lr, momentum=0.99)


    DNN=Sequential()
    act_func=tfunc
    #,kernel_constraint=maxnorm(3)
    DNN.add(Dense(sl1,input_dim=len(inputs),activation=act_func,kernel_regularizer=reg))
    DNN.add(BatchNormalization())
    DNN.add(Dropout(rate=dropout))
    

    if sl2!=0:
        DNN.add(Dense(sl2,activation=act_func,kernel_regularizer=reg))
        DNN.add(BatchNormalization())
        DNN.add(Dropout(rate=dropout))
        
    if sl3!=0:
        DNN.add(Dense(sl3,activation=act_func,kernel_regularizer=reg))
        DNN.add(BatchNormalization())
        DNN.add(Dropout(rate=dropout))
        
    if sl4!=0:
        DNN.add(Dense(sl4,activation=act_func))
        DNN.add(BatchNormalization())
        DNN.add(Dropout(rate=dropout))
        
    if sl5!=0:
        DNN.add(Dense(sl5,activation=act_func))
        DNN.add(BatchNormalization())
        DNN.add(Dropout(rate=dropout))
        


    DNN.add(Dense(1,activation='sigmoid'))
    #DNN.add(BatchNormalization())
    #tp= tf.keras.metrics.TruePositives(thresholds =SoB_thr, name= 'tp')
    #fp= tf.keras.metrics.FalsePositives(thresholds =SoB_thr, name= 'fp')
    auc=tf.keras.metrics.AUC()
    DNN.compile(loss=loss,optimizer=opt ) #, metrics=[ auc]'auc [ auc, tp, fp]
    
    return DNN, NN_name

In [7]:
import bayes_opt
import tensorflow as tf
from keras.layers import Dense, Dropout, LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
from keras.constraints import maxnorm
import keras.backend as k
from tensorflow.keras import regularizers
from sklearn.metrics import roc_curve, auc


def Score(roc_auc, roc_auc_train):
    alpha=1
    return roc_auc-alpha*(np.abs(roc_auc-roc_auc_train))
    #function to optimize in optimization
    
def Score_dnn(n_layer, size_layer, lr, l2, dropout):
    l1=0
    #l2=0.001
    #dropout=0.2
    n_layer=int(n_layer)
    size_layer=int(size_layer)
    sl1=size_layer
    sl2,sl3,sl4,sl5 = 0,0,0,0
    if n_layer >= 2:
        sl2=size_layer
    if n_layer >= 3:
        sl3=size_layer
    if n_layer >= 4:
        sl4=size_layer
    if n_layer >= 5:
        sl5=size_layer
    
    base_dir='D:\\Travail\\ZV_analysis\\'
    print('training network with [{}_{}_{}_{}_{}] neurons'.format(sl1,sl2,sl3,sl4,sl5))
    DNN, res_dir=build_dnn(base_dir, lr, l1, l2, dropout, sl1, sl2, sl3, sl4, sl5)
    
    es_callback = EarlyStopping(monitor='val_loss', patience=early_stop, restore_best_weights=True)
    history=DNN.fit(x=X_train, y=y_train['signal'].to_numpy(),
                    sample_weight=y_train['w'].to_numpy(), epochs=epochs, batch_size=batchsize, 
                    verbose=0, callbacks=[es_callback], 
                    validation_data=(X_test, y_test['signal'].to_numpy(),y_test['w'].to_numpy()))

    #test_loss, test_acc = DNN.evaluate(X_test, y_test['signal'], verbose=0)
    #test = DNN.evaluate(X_test, y_test['signal'], verbose=0)
    #print(test)
    #print('\nTest accuracy:', test_acc)

    predictions_NN= DNN.predict(X_test)
    predictions_NN_train=DNN.predict(X_train)
    fpr, tpr, threshold = roc_curve(y_test['signal'].astype('int32'),predictions_NN, pos_label=1, sample_weight=y_test['w'])
    tpr.sort()
    fpr.sort()
    roc_auc =auc(fpr, tpr)
    fpr, tpr, threshold = roc_curve(y_train['signal'].astype('int32'),predictions_NN_train, pos_label=1, sample_weight=y_train['w'])
    tpr.sort()
    fpr.sort()
    roc_auc_train =auc(fpr, tpr)
    return Score(roc_auc, roc_auc_train)

In [8]:
#optimize lr, l1,l2, dropout, sls

epochs=200
batchsize=1024
tfunc='relu'
loss='binary_crossentropy'
#to add parameters to optimize, include them in pbounds, and put them as inputs to Score DNN
pbounds={'n_layer': (1, 5),
         'size_layer': (40, 300), 
         'lr': (0.0001, 0.001),
         'l2' : (0.0001, 0.1),
         'dropout' : (0,0.5),
        }

optimizer = bayes_opt.BayesianOptimization(
    f=Score_dnn,
    pbounds=pbounds,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

In [9]:
# init points = number of random points, n_iter = number of bayesian optimization
optimizer.maximize(init_points=30, n_iter=45)
optimizer.max

|   iter    |  target   |  dropout  |    l2     |    lr     |  n_layer  | size_l... |
-------------------------------------------------------------------------------------
training network with [78_78_0_0_0] neurons
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

|  1        |  0.8338   |  0.2085   |  0.07206  |  0.000100 |  2.209    |  78.16    |
training network with [180_180_0_0_0] neurons
|  2        |  0.8325   |  0.04617  |  0.01871  |  0.000411 |  2.587    |  180.1    |
training network with [47_47_47_47_0] neurons
|  3        |  0.8278   |  0.2096   |  0.06855  |  0.000284 |  4.512    |  47.12    |
training network with [91_0_0_0_0] neurons
|  4        |  0.8217   |  0.3352   |  0.04179  |  0.000602 |  1.562    |  91.51    |
training network with [267_267_267_0_0] neurons
|  5        |  0.8166   |  0.4004   |  0.09683  |  0.000382 |  3.769    |  267.9 

|  39       |  0.8229   |  0.07042  |  0.02676  |  0.000865 |  2.796    |  224.7    |
training network with [161_161_161_0_0] neurons
|  40       |  0.8391   |  0.2345   |  0.003862 |  0.000217 |  3.823    |  161.8    |
training network with [56_0_0_0_0] neurons
|  41       |  0.8334   |  0.1923   |  0.05328  |  0.000127 |  1.535    |  56.06    |
training network with [121_121_121_121_0] neurons
|  42       |  0.8216   |  0.1341   |  0.05409  |  0.000508 |  4.911    |  121.6    |
training network with [226_226_0_0_0] neurons
|  43       |  0.8305   |  0.08433  |  0.07504  |  0.000267 |  2.96     |  226.4    |
training network with [124_124_124_0_0] neurons
|  44       |  0.818    |  0.3329   |  0.05257  |  0.000479 |  3.288    |  124.1    |
training network with [229_229_229_229_0] neurons
|  45       |  0.8129   |  0.2653   |  0.05771  |  0.000893 |  4.349    |  230.0    |
training network with [276_276_276_0_0] neurons
|  46       |  0.826    |  0.04041  |  0.02673  |  0.000529 |  3.

KeyboardInterrupt: 